In [1]:
#1) Import all necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import ee
import geemap

In [2]:
#ee.Authenticate()
ee.Initialize()

In [53]:
Map = geemap.Map(center=[39.5, -122], zoom=7.2)
Map

Map(center=[39.5, -122], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

In [37]:
counties = ee.FeatureCollection("TIGER/2018/Counties")
Map.addLayer(counties, {}, 'US Counties')

In [38]:
Map.draw_features

[]

In [39]:
roi = ee.FeatureCollection(Map.draw_features)
selected_counties = counties.filterBounds(roi)
Map.addLayer(selected_counties, {}, "Selected Counties")
Map.centerObject(selected_counties, zoom = 8);

In [40]:
Map.remove_last_drawn()
Map.remove_ee_layer('US Counties')

In [867]:
#c_polygon = selected_counties.geometry().geometries().filter(ee.Filter.hasType('item','Polygon')); 

In [868]:
#geometry = ee.Geometry.MultiPolygon(c_polygon)                     
#geometry = ee.FeatureCollection(geometry)                                  


In [869]:
#geemap.ee_to_shp(geometry, filename='../downloads/selected_counties.shp')

In [41]:
# Start of fire August 16
#End of fire November 12
prefire_start = '2020-07-15';   
prefire_end = '2020-08-15';


postfire_start = '2020-11-13';
postfire_end = '2020-12-13';
imagery = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

prefireImCol = ee.ImageCollection(imagery.filterDate(prefire_start, prefire_end).filterBounds(selected_counties));
postfireImCol = ee.ImageCollection(imagery.filterDate(postfire_start, postfire_end).filterBounds(selected_counties));

In [42]:
def maskL8sr(image):
  #Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = 1 << 3;
    cloudsBitMask = 1 << 5;
    snowBitMask = 1 << 4;
 #Get the pixel QA band.
    qa = image.select('pixel_qa');
 #All flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).bitwiseAnd(cloudsBitMask).eq(0).bitwiseAnd(snowBitMask).eq(0);
 #Return the masked image, scaled to TOA reflectance, without the QA bands.
    return image.updateMask(mask).select("B[0-9]*").copyProperties(image, ["system:time_start"]);


In [43]:
# Apply platform-specific cloud mask
prefire_CM_ImCol = prefireImCol.map(maskL8sr);
postfire_CM_ImCol = postfireImCol.map(maskL8sr);

In [44]:
pre_mos = prefireImCol.mosaic().clip(selected_counties);
post_mos = postfireImCol.mosaic().clip(selected_counties);

pre_cm_mos = prefire_CM_ImCol.mosaic().clip(selected_counties);
post_cm_mos = postfire_CM_ImCol.mosaic().clip(selected_counties);

In [45]:
preNBR = pre_cm_mos.normalizedDifference(['B5', 'B7']);
postNBR = post_cm_mos.normalizedDifference(['B5', 'B7']);

In [46]:
dNBR_unscaled = preNBR.subtract(postNBR);

#Scale product to USGS standards
dNBR = dNBR_unscaled.multiply(1000);

In [54]:
#visualization parameters for true color images
vis = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 4000, 'gamma': 1.5};
# Add the true color images to the map.
Map.addLayer(pre_mos, vis,'Pre-fire image');
Map.addLayer(post_mos, vis,'Post-fire image');

In [55]:
#Add the true color images to the map.
Map.addLayer(pre_cm_mos, vis,'Pre-fire True Color Image - Clouds masked');
Map.addLayer(post_cm_mos, vis,'Post-fire True Color Image - Clouds masked');

In [56]:
#Burn Ratio Product - Greyscale
grey = ['white', 'black'];
#display pre- and post-fire NBR seperately
Map.addLayer(preNBR, {'min': -1, 'max': 1, 'palette': grey}, 'Prefire Normalized Burn Ratio');
Map.addLayer(postNBR, {'min': -1, 'max': 1, 'palette': grey}, 'Postfire Normalized Burn Ratio');

Map.addLayer(dNBR, {'min': -1000, 'max': 1000, 'palette': grey}, 'dNBR greyscale');


EEException: Image.normalizedDifference: No band named 'B5'. Available band names: [].

In [17]:
#Define an SLD style of discrete intervals to apply to the image.
sld_intervals = '<RasterSymbolizer>' + '<ColorMap type="intervals" extended="false" >' + '<ColorMapEntry color="#ffffff" quantity="-500" label="-500"/>' + '<ColorMapEntry color="#7a8737" quantity="-250" label="-250" />' + '<ColorMapEntry color="#acbe4d" quantity="-100" label="-100" />' + '<ColorMapEntry color="#0ae042" quantity="100" label="100" />' + '<ColorMapEntry color="#fff70b" quantity="270" label="270" />' + '<ColorMapEntry color="#ffaf38" quantity="440" label="440" />' + '<ColorMapEntry color="#ff641b" quantity="660" label="660" />' + '<ColorMapEntry color="#a41fd6" quantity="2000" label="2000" />' + '</ColorMap>' + '</RasterSymbolizer>';

In [18]:
Map.addLayer(dNBR.sldStyle(sld_intervals), {}, 'dNBR classified');

EEException: Image.normalizedDifference: No band named 'B5'. Available band names: [].

In [19]:
thresholds = ee.Image([-1000, -251, -101, 99, 269, 439, 659, 2000]);
classified = dNBR.lt(thresholds).reduce('sum').toInt();

In [20]:
classified.getInfo()

EEException: Image.normalizedDifference: No band named 'B5'. Available band names: [].

In [21]:
#count number of pixels in entire layer
allpix = classified.updateMask(classified); #mask the entire layer

In [22]:
allpix.getInfo()

EEException: Image.normalizedDifference: No band named 'B5'. Available band names: [].

In [893]:
#Map.set_plot_options(add_marker_cluster=True, marker=None)
#Map.roi_reducer = ee.Reducer.count()

#print(Map.roi_reducer)

ee.Reducer({
  "functionInvocationValue": {
    "functionName": "Reducer.count",
    "arguments": {}
  }
})


In [23]:
pixstats = allpix.reduceRegion({
    'reducer': ee.Reducer.count(),               # count pixels in a single class
    'geometry': sel_counties,
    'scale': 30});

NameError: name 'sel_counties' is not defined

In [24]:
allpixels = ee.Number(pixstats.get('sum')); # extract pixel count as a number

NameError: name 'pixstats' is not defined

In [25]:
#create an empty list to store area values in
arealist = [];

In [26]:
# create a function to derive extent of one burn severity class
#arguments are class number and class name
def areacount(cnr, name):
    singleMask =  classified.updateMask(classified.eq(cnr)); # mask a single class // count pixels in a single class
    stats = singleMask.reduceRegion({'reducer': ee.Reducer.count(), 'geometry': area,'scale': 30});
    pix =  ee.Number(stats.get('sum'));
    hect = pix.multiply(900).divide(10000);               #Landsat pixel = 30m x 30m --> 900 sqm
    perc = pix.divide(allpixels).multiply(10000).round().divide(100);   # get area percent by class and round to 2 decimals
    arealist.push({Class: name, Pixels: pix, Hectares: hect, Percentage: perc});

# severity classes in different order
names2 = ['NA', 'High Severity', 'Moderate-high Severity',
'Moderate-low Severity', 'Low Severity','Unburned', 'Enhanced Regrowth, Low', 'Enhanced Regrowth, High'];

In [51]:
#execute function for each class

for i = 0, i < 8, i++:
    {areacount(i, names2[i]);}

print('Burned Area by Severity Class', arealist, '--> click list objects for individual classes');

SyntaxError: invalid syntax (2469732436.py, line 3)